# Factorizer Game for Quadratic $x^2 - 12x + 35$

#### All the imports (and an `algod` as well)

In [4]:
from io import StringIO
from itertools import product
import pandas as pd
import plotly.graph_objects as go

from graviton.blackbox import DryRunExecutor
from tests.clients import get_algod
algod = get_algod()

#### Logic Signature for Quadratic

In [5]:
TEAL = """#pragma version 5
intcblock 1 12 35
pushbytes 0x43616e20796f7520666163746f722031202a20785e32202d203132202a2078202b203335203f // "Can you factor 1 * x^2 - 12 * x + 35 ?"
pop
intc_0 // 1
intc_1 // 12
intc_2 // 35
arg 0
btoi
callsub sub0
store 0
intc_0 // 1
intc_1 // 12
intc_2 // 35
arg 1
btoi
callsub sub0
store 1
load 0
load 1
+
store 2
load 2
callsub sub1
store 3
txn TypeEnum
intc_0 // pay
==
txn CloseRemainderTo
global ZeroAddress
==
&&
arg 0
btoi
arg 1
btoi
!=
&&
load 3
&&
txn Amount
load 3
==
&&
return
sub0: // root_closeness
store 7
store 6
store 5
store 4
load 4
load 7
*
load 7
*
load 6
+
store 8
load 5
load 7
*
store 9
load 8
load 9
<
bnz sub0_l2
load 8
load 9
-
b sub0_l3
sub0_l2:
load 9
load 8
-
sub0_l3:
retsub
sub1: // calculate_prize
store 10
load 10
intc_0 // 1
+
pushint 20 // 20
<
bnz sub1_l2
pushint 0 // 0
b sub1_l3
sub1_l2:
pushint 1000000 // 1000000
pushint 10 // 10
load 10
intc_0 // 1
+
pushint 2 // 2
/
-
*
sub1_l3:
retsub"""

## Let's Dry-Run this Logic Sig as a Contract Account

### To run a proper simulation we'll need to also provide the `pymnt` transaction which the contract approves

In [6]:
# The prize winning args (i.e. the roots of x^2 - 12x + 35)
args = (5,7)

# Payment txn information:
pymt = {"amt": 10_000_000} 
inspector = DryRunExecutor.dryrun_logicsig(algod, TEAL, args, **pymt)

### We get a `DryRunInspector` object:

In [7]:
print(f"""
{type(inspector)=}
{inspector.passed()=}
{inspector.stack_top()=}
{inspector.final_scratch()=}
{inspector.messages()=}
{inspector.max_stack_height()=}
{inspector.status()=}
{inspector.last_log()=}
""")


type(inspector)=<class 'graviton.blackbox.DryRunInspector'>
inspector.passed()=True
inspector.stack_top()=1
inspector.final_scratch()={3: 10000000, 4: 1, 5: 12, 6: 35, 7: 7, 8: 84, 9: 84}
inspector.messages()=['PASS']
inspector.max_stack_height()=4
inspector.status()='PASS'
inspector.last_log()=None



## Generate CSV from a Bunch of Inputs

### But since we're using this to validate a payment, and the amount has a complicated formula (the closer you are to the actual answer, the more you get payed), we need to have the formulas ready:

In [8]:
def payment_amount(p, q):
    if p == q:
        return 0
    return 1_000_000 * max(10 - (sum(map(lambda x: abs(x**2 - 12 * x + 35), (p, q))) + 1) // 2, 0)

### Now let's execute 400 dry runs on all `int` pairs $(x, y)$ in $[0,19] \times [0,19]$

In [9]:
inputs = list(product(range(20), range(20)))
amts = list(map(lambda args: payment_amount(*args), inputs))
dryrun_results, txns = [], []
for args, amt in zip(inputs, amts):
    txn = {"amt": amt}
    txns.append(txn)
    dryrun_results.append(DryRunExecutor.dryrun_logicsig(algod, TEAL, args, **txn))

### Let's create a CSV-report, and since we're in a Notebook, might as well load into a `DataFrame`

In [10]:
csv = inspector.csv_report(inputs, dryrun_results, txns)
df = pd.read_csv(StringIO(csv))
df

,Run,Status,cost,final_message,last_log,top_of_stack,Arg_00,Arg_01,amt,max_stack_height,...,s@002,s@003,s@004,s@005,s@006,s@007,s@008,s@009,s@010,steps
0,1,REJECT,NaN,REJECT,`None,0,0,0,0,4,...,70.0,NaN,1,12,35,NaN,35,NaN,70.0,105
1,2,REJECT,NaN,REJECT,`None,0,0,1,0,4,...,59.0,NaN,1,12,35,1.0,36,12.0,59.0,105
2,3,REJECT,NaN,REJECT,`None,0,0,2,0,4,...,50.0,NaN,1,12,35,2.0,39,24.0,50.0,105
3,4,REJECT,NaN,REJECT,`None,0,0,3,0,4,...,43.0,NaN,1,12,35,3.0,44,36.0,43.0,105
4,5,REJECT,NaN,REJECT,`None,0,0,4,0,4,...,38.0,NaN,1,12,35,4.0,51,48.0,38.0,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,396,REJECT,NaN,REJECT,`None,0,19,15,0,4,...,248.0,NaN,1,12,35,15.0,260,180.0,248.0,105
396,397,REJECT,NaN,REJECT,`None,0,19,16,0,4,...,267.0,NaN,1,12,35,16.0,291,192.0,267.0,105
397,398,REJECT,NaN,REJECT,`None,0,19,17,0,4,...,288.0,NaN,1,12,35,17.0,324,204.0,288.0,105
398,399,REJECT,NaN,REJECT,`None,0,19,18,0,4,...,311.0,NaN,1,12,35,18.0,359,216.0,311.0,105


### Some cleanup

In [11]:
df = (df[[col for col in df.columns if col not in (" Run", " cost", " last_log")]]
    .rename({
        "Arg_00": "x",
        "Arg_01": "y",
    }, axis=1)
).fillna(0)
df

,Status,final_message,top_of_stack,x,y,amt,max_stack_height,s@000,s@001,s@002,s@003,s@004,s@005,s@006,s@007,s@008,s@009,s@010,steps
0,REJECT,REJECT,0,0,0,0,4,35.0,35.0,70.0,0.0,1,12,35,0.0,35,0.0,70.0,105
1,REJECT,REJECT,0,0,1,0,4,35.0,24.0,59.0,0.0,1,12,35,1.0,36,12.0,59.0,105
2,REJECT,REJECT,0,0,2,0,4,35.0,15.0,50.0,0.0,1,12,35,2.0,39,24.0,50.0,105
3,REJECT,REJECT,0,0,3,0,4,35.0,8.0,43.0,0.0,1,12,35,3.0,44,36.0,43.0,105
4,REJECT,REJECT,0,0,4,0,4,35.0,3.0,38.0,0.0,1,12,35,4.0,51,48.0,38.0,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,REJECT,REJECT,0,19,15,0,4,168.0,80.0,248.0,0.0,1,12,35,15.0,260,180.0,248.0,105
396,REJECT,REJECT,0,19,16,0,4,168.0,99.0,267.0,0.0,1,12,35,16.0,291,192.0,267.0,105
397,REJECT,REJECT,0,19,17,0,4,168.0,120.0,288.0,0.0,1,12,35,17.0,324,204.0,288.0,105
398,REJECT,REJECT,0,19,18,0,4,168.0,143.0,311.0,0.0,1,12,35,18.0,359,216.0,311.0,105


### Zoom in on solution $(x, y) = (5, 7)$:

In [12]:
df[(df.x == 5) & (df.y == 7)][[" Status", " top_of_stack", "x", "y", "amt"] + [f"s@00{i}" for i in range(0,10)]]

,Status,top_of_stack,x,y,amt,s@000,s@001,s@002,s@003,s@004,s@005,s@006,s@007,s@008,s@009
107,PASS,1,5,7,10000000,0.0,0.0,0.0,10000000.0,1,12,35,7.0,84,84.0


### Looks like `Scratch Slot 3` is where the expected payment amount is stored (except when the program `REJECT`s):

In [13]:
assert all((df[" Status"] == "REJECT") | (df.amt == df["s@003"]))

## 3D-Plots from our Blackbox Results

In [14]:
def inspectors_3D_from_dryruns(max_x, max_y):
    X = list(range(max_x))
    Y = list(range(max_y))

    Z = []
    for x in X:
        row = []
        for y in Y:
            row.append(DryRunExecutor.dryrun_logicsig(algod, TEAL, (x, y), amt=payment_amount(x,y)))
        Z.append(row)

    return X, Y, Z

In [15]:
def blackbox_plot(xs, ys, zs, ztick='µA', ztitle='prize (µA)', title='pymt txn analysis'):
    fig = go.Figure(data=[go.Surface(z=zs, x=xs, y=ys)])
    fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                      highlightcolor="limegreen", project_z=True))
    fig.update_layout(title=title, autosize=False,
                      scene=dict(
                        zaxis=dict(ticksuffix=ztick),
                        zaxis_title=ztitle,
                      ),
                      scene_camera_eye=dict(x=1.87, y=0.88, z=-0.64),
                      width=600, height=600,
                      margin=dict(l=65, r=50, b=65, t=90)
                      )                                   
    return fig.show()

In [16]:
X, Y, i3d = inspectors_3D_from_dryruns(21, 21)

In [17]:
blackbox_plot(X, Y, [[inspector.final_scratch().get(3,0) for inspector in row] for row in i3d], title="Scratch Slot #3")

In [18]:
blackbox_plot(X, Y, [[inspector.max_stack_height() for inspector in row] for row in i3d], ztick=None, ztitle="Max Stack Height", title="Max Stack Height Analysis")

In [19]:
blackbox_plot(
    X, 
    Y, 
    [[inspector.final_scratch().get(0,0) for inspector in row] for row in i3d], 
    ztick=None, 
    ztitle="Scratch Slot 0",
    title="Slot 0 Analysis"
)

In [20]:
blackbox_plot(X, Y, [[inspector.stack_top() for inspector in row] for row in i3d], ztick=None, ztitle="final stack top", title="Stack Top Analysis")

# Hackey Appendix

In [21]:
"""
Ok... This is a hack that I came up with last minute for the demo... 
But this could easily be done in a straightforward, non-hacky way (just build z directly from the dry runs)
"""
def x(row):
    return row.x

def y(row):
    return row.y

def z_factory(df, z_col="amt"):
    def z(_x, _y):
        _rows = df[(df.x == _x) & (df.y == _y)]
        assert len(_rows) == 1
        return _rows.iloc[0][z_col]

    return z

z = z_factory(df)

row = df[(df.x == 5) & (df.y == 7)].iloc[0]
z(x(row), y(row))

xs = sorted(df.x.unique())
ys = sorted(df.y.unique())
zs = [[z(_x, _y) for _y in ys] for _x in xs]

In [22]:
blackbox_plot(xs, ys, zs)